# Federated Transfer Learning for Anomaly Detection in HPC Systems
This notebook simulates FTL using a dense autoencoder and TensorFlow Federated.

In [ ]:
!pip install tensorflow tensorflow_federated scikit-learn numpy pandas

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler

## Simulate Data for Nodes

In [ ]:
def generate_node_data(n_samples=500, n_features=462, anomaly_ratio=0.03):
    X = np.random.normal(0, 1, size=(n_samples, n_features))
    y = np.random.choice([0, 1], size=n_samples, p=[1-anomaly_ratio, anomaly_ratio])
    return X, y

client_data = [generate_node_data() for _ in range(5)]

## Define Dense Autoencoder

In [ ]:
def create_autoencoder(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_dim),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(80, activation='relu'),
        tf.keras.layers.Dense(60, activation='relu'),
        tf.keras.layers.Dense(40, activation='relu'),
        tf.keras.layers.Dense(20, activation='relu'),  # latent
        tf.keras.layers.Dense(40, activation='relu'),
        tf.keras.layers.Dense(60, activation='relu'),
        tf.keras.layers.Dense(80, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(input_dim, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-6),
                  loss='mse')
    return model

## Preprocessing and Federated Learning Setup

In [ ]:
def preprocess(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data)

def model_fn():
    return tff.learning.from_keras_model(
        keras_model=create_autoencoder(input_dim=462),
        input_spec=tf.TensorSpec([None, 462], tf.float32),
        loss=tf.keras.losses.MeanSquaredError()
    )

def get_federated_data():
    federated_data = []
    for X, _ in client_data:
        X_scaled = preprocess(X)
        federated_data.append(tf.data.Dataset.from_tensor_slices(X_scaled).batch(10))
    return federated_data

## Train Federated Model (FedAvg)

In [ ]:
federated_averaging = tff.learning.build_federated_averaging_process(model_fn)
state = federated_averaging.initialize()

for round_num in range(10):  # Example: 10 rounds
    state, metrics = federated_averaging.next(state, get_federated_data())
    print(f'Round {round_num+1}, Loss: {metrics.loss}')

## Transfer Learning to Unseen Node

In [ ]:
X_unseen, y_unseen = generate_node_data()
X_unseen_scaled = preprocess(X_unseen)

global_model = create_autoencoder(input_dim=462)
global_model.set_weights(state.model.trainable)

for layer in global_model.layers[:5]:
    layer.trainable = False

global_model.compile(optimizer='RMSprop', loss='mse')
global_model.fit(X_unseen_scaled, X_unseen_scaled, epochs=10, batch_size=10)

## Evaluate on Unseen Node

In [ ]:
reconstructed = global_model.predict(X_unseen_scaled)
recon_error = np.mean((X_unseen_scaled - reconstructed)**2, axis=1)
threshold = 0.5
predictions = (recon_error > threshold).astype(int)

print("F1-Score:", f1_score(y_unseen, predictions))
print("Precision:", precision_score(y_unseen, predictions))
print("Recall:", recall_score(y_unseen, predictions))